# Level 2 - Week 7 - 02 Guardrails

**Estimated time:** 60-90 minutes

## Learning Objectives

- Allowlist tools
- Enforce step caps
- Add timeout and budget caps


## Overview

Guardrails prevent runaway agents and unsafe actions.

Implement in this order:

1. **Tool allowlist** (restrict what actions are possible)
2. **Step cap** (prevent infinite loops)
3. **Timeout cap** (prevent hanging requests)
4. **Budget cap** (limit cost and blast radius)

## Underlying theory: guardrails enforce trust boundaries

An agent is a controller that can cause side effects (tool calls).

Inputs have different trust levels:

- system instructions (high trust)
- user input (medium trust)
- retrieved documents / web pages (low trust)

Guardrails are hard rules that prevent low-trust inputs from escalating into high-impact actions.

### Prompt injection reality

Retrieved text can contain instructions.

Rule:

- retrieved documents are data, not instructions

## Practice Steps

- Define a guardrail config.
- Enforce allowlists + caps before every tool call.
- When a guardrail triggers, stop and return a clear safe outcome.

### Sample code

Guardrail config and checker.


In [ ]:
from dataclasses import dataclass

@dataclass
class Guardrails:
    allowed_tools: list[str]
    max_steps: int


def check_guardrails(tool_name: str, step_index: int, rules: Guardrails) -> None:
    if tool_name not in rules.allowed_tools:
        raise ValueError('tool not allowed')
    if step_index >= rules.max_steps:
        raise ValueError('step cap exceeded')


### Student fill-in

Integrate guardrails into an agent loop.


In [ ]:
from __future__ import annotations

import time
from dataclasses import dataclass
from typing import Callable


@dataclass(frozen=True)
class Guardrails:
    allowed_tools: set[str]
    max_steps: int
    max_runtime_s: float


def check_guardrails(tool_name: str, step_index: int, elapsed_s: float, rules: Guardrails) -> None:
    if tool_name not in rules.allowed_tools:
        raise ValueError("tool_not_allowed")
    if step_index >= rules.max_steps:
        raise ValueError("step_cap_exceeded")
    if elapsed_s > rules.max_runtime_s:
        raise ValueError("timeout_exceeded")


def tool_search(payload: dict) -> dict:
    q = payload.get("query", "")
    return {"hits": [{"chunk_id": "kb#001", "score": 0.82, "text": f"hit for {q}"}]}


def tool_write_answer(payload: dict) -> dict:
    hits = payload.get("hits", [])
    if not hits:
        return {"mode": "clarify", "answer": None, "citations": []}
    return {"mode": "answer", "answer": "stub", "citations": [{"chunk_id": hits[0]["chunk_id"]}]}


def run_agent_with_guardrails(task: str, plan: list[str], tools: dict[str, Callable[[dict], dict]], rules: Guardrails) -> dict:
    t0 = time.perf_counter()
    steps: list[dict] = []

    for step_index, tool_name in enumerate(plan, start=1):
        elapsed = time.perf_counter() - t0
        try:
            check_guardrails(tool_name, step_index=step_index, elapsed_s=elapsed, rules=rules)
        except Exception as e:
            return {"mode": "refuse", "reason": str(e), "steps": steps}

        if tool_name == "search":
            out = tools[tool_name]({"query": task, "top_k": 3})
        elif tool_name == "write_answer":
            hits = next((s["output"].get("hits", []) for s in steps if s["tool"] == "search"), [])
            out = tools[tool_name]({"question": task, "hits": hits})
        else:
            out = tools[tool_name]({})

        steps.append({"tool": tool_name, "output": out})

    return {"mode": "answer", "steps": steps}


tools = {"search": tool_search, "write_answer": tool_write_answer}
rules = Guardrails(allowed_tools={"search", "write_answer"}, max_steps=3, max_runtime_s=5.0)

print(run_agent_with_guardrails("refund policy", plan=["search", "write_answer"], tools=tools, rules=rules))
print(run_agent_with_guardrails("refund policy", plan=["search", "delete_files"], tools=tools, rules=rules))

## Self-check

- Is the allowlist enforced?
- Are steps capped?
